In [1]:
# from google.colab import files

# uploaded = files.upload()

import ssl 
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import pandas as pd

df = pd.read_csv('train_comment_small_50.csv', sep=',')

In [6]:
import re

def clean_comment(text):
    # Strip HTML tags
    text = re.sub('<[^<]+?>', ' ', text)
 
    # Strip escaped quotes
    text = text.replace('\\"', '')
 
    # Strip quotes
    text = text.replace('"', '')
    
    # 去掉换行
    text = text.replace("\n","")
 
    return text

In [7]:
df["comment_text"]

0     "\n \n I'm sure if you worked on articles othe...
1     D'aww! He matches this background colour I'm s...
2     Hey man, I'm really not trying to edit war. It...
3     "\n More\n I can't make any real suggestions o...
4      REDIRECT Talk:Voydan Pop Georgiev- Chernodrinski
5     The Mitsurugi point made no sense - why not ar...
6     Don't mean to bother you \n \n I see that you'...
7     "\n \n  Regarding your recent edits \n \n Once...
8     "\n Good to know. About me, yeah, I'm studying...
9     "\n \n  Snowflakes are NOT always symmetrical!...
10    "\n \n  The Signpost: 24 September 2012 \n \n ...
11    "\n \n Re-considering 1st paragraph edit?\n I ...
12    Bye! \n \n Don't look, come or think of commin...
13    How could I post before the block expires? The...
14    Not sure about a heading of 'Fight for Freedom...
15    Praise \n \n looked at this article about 6 mo...
16    I was able to post the above list so quickly b...
17    "\n Well, not ""before the process"" but "

In [5]:
df

,comment_text,toxic
0,"""\n \n I'm sure if you worked on articles othe...",0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\n More\n I can't make any real suggestions o...",0
4,REDIRECT Talk:Voydan Pop Georgiev- Chernodrinski,0
5,The Mitsurugi point made no sense - why not ar...,0
6,Don't mean to bother you \n \n I see that you'...,0
7,"""\n \n Regarding your recent edits \n \n Once...",0
8,"""\n Good to know. About me, yeah, I'm studying...",0
9,"""\n \n Snowflakes are NOT always symmetrical!...",0


In [8]:
from sklearn.model_selection import train_test_split

df['cleaned_comment'] = df['comment_text'].apply(clean_comment)
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_comment'], df['toxic'], test_size=0.2)

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wujiangu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
 
vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'), 
                             lowercase=True, min_df=3, max_df=0.9, max_features=5000)
X_train_onehot = vectorizer.fit_transform(X_train)

In [14]:
from keras.models import Sequential
from keras.layers import Dense
 
nn = Sequential()   #初始化神经网络
 

# 添加隐含层
nn.add(Dense(units=500, activation='relu', input_dim=len(vectorizer.get_feature_names())))

# 添加输出层
nn.add(Dense(units=1, activation='sigmoid'))
 
    
# 二进制交叉熵作为损失函数，将使用Adam算法（是随机梯度下降的扩展）
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 500)               31500     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 501       
Total params: 32,001
Trainable params: 32,001
Non-trainable params: 0
_________________________________________________________________


In [15]:
nn.fit(X_train_onehot[:-20], y_train[:-20], 
          epochs=5, batch_size=128, verbose=1, 
          validation_data=(X_train_onehot[-20:], y_train[-20:]))

Epoch 1/5


InvalidArgumentError:  TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 242, in __call__
    return func(device, token, args)

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 131, in __call__
    ret = self._func(*args)

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 302, in wrapper
    return func(*args, **kwargs)

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 513, in py_method
    return [slice_array(inp) for inp in flat_inputs]

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py"

In [16]:
scores = nn.evaluate(vectorizer.transform(X_test), y_test, verbose=1)
print("Accuracy:", scores[1]) 

InvalidArgumentError:  TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 242, in __call__
    return func(device, token, args)

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 131, in __call__
    ret = self._func(*args)

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 302, in wrapper
    return func(*args, **kwargs)

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 513, in py_method
    return [slice_array(inp) for inp in flat_inputs]

  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py"

In [ ]:
nn.save('nn.hd5')